In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/i550012/virtualenv/tensorflow1

In [7]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']

ratings_df = pd.read_csv('./movielens/ml-100k/u.data', sep='\t', names=columns)
ratings_pt = ratings_df.pivot(index='user_id', columns='item_id', values='rating').fillna(0)

train_raitings_df = ratings_df[:80000]
test_raitings_df = ratings_df[80000:]

user_indices_train = [i-1 for i in train_raitings_df.user_id.values]
item_indices_train = [i-1 for i in train_raitings_df.item_id.values]
R_known_train = train_raitings_df.rating.values

user_indices_test = [i-1 for i in train_raitings_df.user_id.values]
item_indices_test = [i-1 for i in train_raitings_df.item_id.values]
R_known_test = train_raitings_df.rating.values
  

In [10]:
R = ratings_pt
N = 943
M = 1682
K = 10
p = np.random.rand(N, K)
q = np.random.rand(K, M)

In [11]:
# prediction ^R = p*q
P = tf.Variable(p, dtype=tf.float32, name='P')
Q = tf.Variable(q, dtype=tf.float32, name='Q')
R_hat = tf.matmul(P, Q)

R_hat_flatten = tf.reshape(R_hat, [-1])
R_hat_train_known = tf.gather(R_hat_flatten, user_indices_train*tf.shape(R_hat)[1] 
                              + item_indices_train, name='extract_user_ratings')

In [12]:
 # total cost sum||r_hat -r|| + lambda(||p|| + ||q||)
 diff_ratings = tf.subtract(R_hat_train_known, R_known_train, name='diff_raitings')
 diff_ratings_abs = tf.abs(diff_ratings, name='abs_error')
 base_cost = tf.reduce_sum(diff_ratings_abs, name='sum_abs_error')
 lda = tf.constant(.001, name='lambda')
 norm_sums = tf.add(tf.reduce_sum(tf.abs(P, name='user_abs'), name='user_norm'),
                    tf.reduce_sum(tf.abs(Q, name='item_abs'), name='item_norm'))
 regularizer = tf.multiply(norm_sums, lda, 'regularizer')
 cost = tf.add(base_cost, regularizer)

In [13]:
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(cost)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for i in range(500):
    sess.run(train)

2024-04-19 05:18:19.854614: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


In [17]:
R_hat_test_known = tf.gather(R_hat_flatten, user_indices_test*tf.shape(R_hat)[1] + item_indices_test, name='extract_user_ratings')
diff_ratings_test = tf.subtract(R_hat_test_known, R_known_test, name='diff_ratings_test')
diff_ratings_test_abs = tf.abs(diff_ratings_test, name='abs_error_test')
sum_diff_ratings_test_abs = tf.reduce_sum(diff_ratings_test_abs, name='sum_abs_error_test')
accuracy = tf.div(sum_diff_ratings_test_abs, R_known_test.size, name='accuracy')
print(sess.run(accuracy))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
0.5540129


In [18]:
R.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
pred = sess.run(R_hat)
pred_df = pd.DataFrame(pred)
pred_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,4.022923,2.935396,1.384603,3.062318,2.932747,4.593189,4.053580,4.705018,4.275819,2.761296,...,2.670622,2.810325,1.579049,3.128966,2.774600,1.494040,3.230643,3.728130,2.452745,3.269543
1,4.076972,3.626112,2.766515,3.301582,4.075958,4.847106,4.370744,4.389758,4.876236,4.000603,...,2.586519,2.489247,1.962374,3.299466,3.548034,1.462035,2.956035,3.921902,2.554857,2.983485
2,2.807352,2.406153,2.959936,3.963768,1.842296,3.967366,2.605143,2.538274,2.429245,2.335020,...,1.880644,2.341130,1.306009,2.156262,1.788317,2.015703,1.960186,1.013944,1.844748,2.520568
3,4.819562,4.299984,3.401508,3.074560,4.548990,5.539420,4.862272,5.282519,4.793284,3.887272,...,3.291537,3.594764,1.871719,3.042090,3.855801,1.809662,3.727893,4.020696,2.844139,4.033352
4,3.598917,2.781453,1.468440,1.686373,2.333752,4.765517,3.111234,3.787403,3.190113,2.799037,...,2.105472,2.705868,1.272904,2.664420,2.867555,2.078190,2.855243,1.829949,3.065022,3.731499
